In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random

In [5]:
import pandas as pd

file_path = 'D:/AA Praktyki/df_parties.csv'
df = pd.read_csv(file_path)
df=df[['id','role','name','city_by_all']]
duplicate_check = df[df['role'].isin(['APPLICANTS', 'ASSIGNEE', 'INVENTORS'])]
duplicate_group = duplicate_check.groupby('id')['name'].nunique().reset_index()
ids_to_remove = duplicate_group[duplicate_group['name'] == 1]['id']
result_df = df[~df['id'].isin(ids_to_remove)]
result_df = df[df['role'].isin(['APPLICANTS', 'ASSIGNEE'])]
result_df=result_df.head(10)

print(result_df)

          id        role                                               name  \
0   P.277022  APPLICANTS  WARSZAWSKIE CENTRUM POSTĘPU TECHNICZNO-ORGANIZ...   
1   P.277022    ASSIGNEE  WARSZAWSKIE CENTRUM POSTĘPU TECHNICZNO-ORGANIZ...   
6   P.277023  APPLICANTS  WARSZAWSKIE CENTRUM POSTĘPU TECHNICZNO-ORGANIZ...   
7   P.277023    ASSIGNEE  WARSZAWSKIE CENTRUM POSTĘPU TECHNICZNO-ORGANIZ...   
9   P.277024  APPLICANTS  WARSZAWSKIE CENTRUM POSTĘPU TECHNICZNO-ORGANIZ...   
10  P.277024    ASSIGNEE  WARSZAWSKIE CENTRUM POSTĘPU TECHNICZNO-ORGANIZ...   
12  P.277025  APPLICANTS  BIURO PROJEKTÓW BUDOWNICTWA KOMUNALNEGO "STOLICA"   
13  P.277025    ASSIGNEE  BIURO PROJEKTÓW BUDOWNICTWA KOMUNALNEGO "STOLICA"   
15  P.277026  APPLICANTS                                SOKOŁOWSKI MIROSŁAW   
16  P.277026    ASSIGNEE                                SOKOŁOWSKI MIROSŁAW   

   city_by_all  
0     WARSZAWA  
1     WARSZAWA  
6     WARSZAWA  
7     WARSZAWA  
9     WARSZAWA  
10    WARSZAWA  
12    WARSZ

In [20]:
import pandas as pd 

df = pd.read_csv('D:/AA Praktyki/df_parties.csv')
df = df[['id', 'role', 'name', 'city_by_all']]
check = df[df['role'].isin(['APPLICANTS', 'ASSIGNEE', 'INVENTORS'])]
ids_with_inventor = check[check['role'] == 'INVENTORS']['id'].unique()
duplicate_group = check.groupby('id')['name'].nunique().reset_index()
ids_to_remove = duplicate_group[(duplicate_group['name'] == 1) & (duplicate_group['id'].isin(ids_with_inventor))]['id']
result = df[~df['id'].isin(ids_to_remove) & df['role'].isin(['APPLICANTS', 'ASSIGNEE'])]
result = result.drop_duplicates(subset='name', keep='first')
result.to_csv('D:/AA Praktyki/df_parties_without_duplicates.csv', index=False)

print(result.head())


          id        role                                               name  \
0   P.277022  APPLICANTS  WARSZAWSKIE CENTRUM POSTĘPU TECHNICZNO-ORGANIZ...   
12  P.277025  APPLICANTS  BIURO PROJEKTÓW BUDOWNICTWA KOMUNALNEGO "STOLICA"   
18  P.277027  APPLICANTS                           INSTYTUT ELEKTROTECHNIKI   
28  P.277029  APPLICANTS  CENTRUM TECHNIKI URZĄDZEŃ TRANSPORTU BLISKIEGO...   
34  P.277030  APPLICANTS  CENTRUM TECHNIKI URZĄDZEŃ TRANSPORTU BLISKIEGO...   

         city_by_all  
0           WARSZAWA  
12          WARSZAWA  
18          WARSZAWA  
28   GDAŃSK-WRZESZCZ  
34  GDAŃSK- WRZESZCZ  


In [2]:
import pandas as pd
import random
import requests
from bs4 import BeautifulSoup
import re

df = pd.read_csv('D:/AA Praktyki/df_parties.csv')
df = df[['id', 'role', 'name', 'city_by_all']]

first_row = df[['id', 'name', 'city_by_all']].iloc[10]
company_id = first_row['id']
company_name = first_row['name']
city_name = first_row['city_by_all']

USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36",
]

headers = {
    "User-Agent": random.choice(USER_AGENTS)
}

query = f"{company_name} {city_name} numer KRS"
search_url = f"https://www.google.com/search?q={query}"

response = requests.get(search_url, headers=headers, timeout=30)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")
results = soup.find_all('div', class_='g')

krs_number = None
for result in results:
    text = result.get_text()
    match = re.search(r"KRS\s*:?(\d{9,10})", text, re.IGNORECASE)
    if match:
        krs_number = match.group(1)
        break

results_df = pd.DataFrame([{
    'id': company_id,
    'name': company_name,
    'city_by_all': city_name,
    'krs': krs_number
}])

print(results_df)


         id                                               name city_by_all  \
0  P.277024  WARSZAWSKIE CENTRUM POSTĘPU TECHNICZNO-ORGANIZ...    WARSZAWA   

          krs  
0  0000090388  


In [4]:
import pandas as pd
import random
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

def configure_driver():
    options = webdriver.ChromeOptions()
    ua = UserAgent()
    options.add_argument(f"user-agent={ua.random}")
    options.add_argument("--headless") 
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--incognito")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

driver = configure_driver()
df = pd.read_csv('D:/AA Praktyki/df_parties_without_duplicates.csv')
df = df[['id', 'name', 'city_by_all']]
def simulate_user_interaction(driver):
    scroll_distance = random.randint(100, 1000)
    driver.execute_script(f"window.scrollBy(0, {scroll_distance});")
    time.sleep(random.uniform(0.5, 2))

def search_krs_in_google(name, city):
    query = f"{name} {city} numer KRS"
    driver.get(f"https://www.google.com/search?q={query}")
    simulate_user_interaction(driver)
    time.sleep(random.uniform(2, 5))

    soup = BeautifulSoup(driver.page_source, "html.parser")
    results = soup.select("div.g")

    krs_number = None
    for result in results:
        text = result.get_text()
        match = re.search(r"KRS\s*:?(\d{9,10})", text, re.IGNORECASE)
        if match:
            krs_number = match.group(1)
            break

    return krs_number
def search_krs_on_bizraport(name):
    driver.get("https://www.bizraport.pl/")
    try:
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[placeholder='Szukaj organizacji, branży lub osoby...']"))
        )
        search_box.send_keys(name)
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label='Szukaj']"))
        )
        search_button.click()
        time.sleep(random.uniform(3, 6))

        soup = BeautifulSoup(driver.page_source, "html.parser")
        krs_number = None

        match = re.search(r"KRS\s*:?(\d{9,10})", soup.get_text(), re.IGNORECASE)
        if match:
            krs_number = match.group(1)
        return krs_number
    except TimeoutException:
        print("Timeout: Element nie został znaleziony na stronie.")
        return None
    except NoSuchElementException:
        print("NoSuchElementException: Nie znaleziono elementu na stronie.")
        return None

results = []

for index, row in df.iterrows():
    company_id = row['id']
    company_name = row['name']
    city_name = row['city_by_all']
    krs_number = search_krs_in_google(company_name, city_name)

    if not krs_number:
        print(f"Numer KRS nie znaleziony w Google dla {company_name}. Przechodzę na bizraport.pl...")
        krs_number = search_krs_on_bizraport(company_name)
    
    results.append({
        'id': company_id,
        'name': company_name,
        'city_by_all': city_name,
        'krs': krs_number
    })

    if (index + 1) % 10 == 0 or (index + 1) == len(df):
        debug_df = pd.DataFrame(results)
        debug_df.to_csv('D:/AA Praktyki/debug_krs_numbers6.csv', index=False)
        print(f"Debug: Zapisano {index + 1} wyników do pliku debugowego.")
results_df = pd.DataFrame(results)
results_df.to_csv('D:/AA Praktyki/krs_numbers.csv', index=False)
print("Dane zostały zapisane do pliku 'krs_numbers.csv'.")
print("\nWyniki wyszukiwania numerów KRS:")
print(results_df)
driver.quit()


Numer KRS nie znaleziony w Google dla KARPIŃSKA MAŁGORZATA-TAPPARELLA. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla EGIS GYORGYSZERGYAR RT.. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla CLOSURE SYSTEMS INTERNATIONAL JAPAN, LIMITED. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla CAVICCHIOLI STEFANO. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla JONES BRIAN C.. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla ROTHENBERG PAUL JOHN. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 35250 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla FLEXIDRILL LIMITED. Przechodzę na 

Numer KRS nie znaleziony w Google dla UNIWERSYTET W BIAŁYMSTOKU. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla RÓJ REMIGIUSZ. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 35420 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla M&S POMORSKA FABRYKA OKIEN SPÓŁKA Z OGRANICZONĄ ODPOWIEDZIALNOŚCIĄ. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla WINOWIECKI MARIAN. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla KRZYŚLAK PIOTR. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 35430 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla TÜSCHEN UND ZIMMERMANN GMBH & CO.KG. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Nu

Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla ZAKŁAD PRODUKCYJNO-HANDLOWO-USŁUGOWY TERMO-TECH SPÓŁKA JAWNA TADEUSZ PILUT WACŁAW PILUT. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 35590 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla JANCZEWSKA-GARDASIEWICZ AGNIESZKA ASC. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla ŚPICA DOROTA. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 35600 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla LEKSIŃSKA MARZENA Q CLUB. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla TRONOX PIGMENTS GMBH. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla ISSA ISSAM. Przechodzę n

Numer KRS nie znaleziony w Google dla WRONKOWSKI CEZARY. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla BIERNACKI MACIEJ TADEUSZ. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 35740 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla MYUNG JIN S.M. CO., LTD.. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla JAŻDŻEWSKI PIOTR PRZEDSIĘBIORSTWO HANDLOWO-USŁUGOWE AMIDA. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla LEKKI WŁODZIMIERZ. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla KORDYL MONIKA LABORATORIUM INNOWACYJNO WDROŻENIOWE LIW. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Goo

Numer KRS nie znaleziony w Google dla INSTYTUT GOSPODARKI SUROWCAMI MINERALNYMI I ENERGIĄ PAN. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 35940 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla BARECKI ZBIGNIEW "ZBIGNIEW BARECKI". Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 35950 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla BURY ANNA. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 35960 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla PAWŁOWICZ MAREK ZAKŁAD BUDOWY MASZYN M. M. OPTIMAT. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla HASENBECK ANTONI ZAKŁAD HANDLOWO-USŁUGOWY HASAN EKSPORT-IMPORT. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Goog

...

Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla CICHOCKI KRZYSZTOF HYDROG ZAKŁAD BUDOWY MASZYN EKSPORT-IMPORT. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 50880 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla RYBIAŁEK JAROSŁAW. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla RYBIAŁEK JAROSŁAW MAREK. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla KUSYK GABRIELA ZAKŁAD PRODUKCYJNO HANDLOWO USŁUGOWY KUSYK SPÓŁKA CYWILNA. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla CZERWIŃSKI MACIEJ STOMATOLOGIA SUCCESS STAIRS. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 50890 wyników do pliku debugowego.
Debug: Zapisano 50900 wy

Debug: Zapisano 51160 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla KNYSAK JAGODA. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 51170 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla WASIAK MAREK NERO SPÓŁKA CYWILNA. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 51180 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla PAJĄK JAN. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 51190 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla UAB "BALTIC CAPS". Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla KLIMAS TOMASZ. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 51200 wyników do pliku debugowego.
Debug: Zapisano 51210 wyników do pliku debugowego.
Numer

Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla KOT TOMASZ PHU INFOCOPY. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 51470 wyników do pliku debugowego.
Debug: Zapisano 51480 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla TEODORCZUK TOMASZ. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla GOŁĘBIOWSKA-MAGDA DOROTA NAVIGATOR24. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla MAGDA KRZYSZTOF FHU. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla CZEŻYK ARTUR. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 51490 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla MRZYGŁÓD JAN KIPERTECH CONSULTING. Przechodzę 

Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla NIEMCZAK MICHAŁ. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 51740 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla MATERIALS DESIGN, SYSTEMS & DEVICES SPÓŁKA Z OGRANICZONĄ ODPOWIEDZIALNOŚCIĄ. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla OLEKSIIOVYCH KONSTIANTYN YEFYMENKO. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla YEFYMENKO KONSTIANTYN OLEKSIIOVYCH. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla PUBLIC JOINT STOCK COMPANY MINING AND METALLURGICAL COMPANY NORILSK NICK. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 51750 wyników do pliku debugowego.
Numer KRS ni

Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 51960 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla CHWIAŁKIEWICZ MACIEJ REYNOX. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla VERED RON. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla WIGHT JAMES STUART. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Debug: Zapisano 51970 wyników do pliku debugowego.
Numer KRS nie znaleziony w Google dla PARISIEN BRIAN RUDY. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla FUNG DAVID TAT FAI. Przechodzę na bizraport.pl...
Timeout: Element nie został znaleziony na stronie.
Numer KRS nie znaleziony w Google dla INSTYTUT BIOLOGII DOŚWIADCZALNEJ IM. MARCELEGO NENCKIEGO POLSKA AKADEMIA NAUK. Przechodzę na bizraport.pl...
Time

In [3]:
import pandas as pd 
df = pd.read_csv('D:/AA Praktyki/df_parties_without_duplicates.csv')
df = df[['id', 'name', 'city_by_all']].iloc[35240:].head()
df

,id,name,city_by_all
35240,P.378685,PIOTROWSKI SŁAWOMIR KOMFORT GROUP,OSIELSKO
35241,P.378685,GAWRYSIAK BOGUSŁAW,BYDGOSZCZ
35242,P.378692,KARPIŃSKA MAŁGORZATA-TAPPARELLA,RADOM
35243,P.378702,FABRYKA ARMATURY HAWLE SPÓŁKA Z OGRANICZONĄ OD...,KOZIEGŁOWY
35244,P.378711,EGIS GYORGYSZERGYAR RT.,BUDAPESZT


In [5]:
df_patents = pd.read_csv('D:/AA Praktyki/df_patents9020.csv')
array=df_patents['id'].unique()
array


array(['P.283166', 'P.283192', 'P.283197', ..., 'P.427944', 'P.428285',
       'P.428779'], dtype=object)

In [7]:
len(array)

21319